In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!kaggle datasets download -d brendan45774/test-file

In [ ]:
!unzip /content/test-file.zip

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Reading csv file

In [8]:
df=pd.read_csv('/content/tested.csv')

columns name

In [9]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [10]:
df.sample(6)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
409,1301,1,3,"Peacock, Miss. Treasteall",female,3.0,1,1,SOTON/O.Q. 3101315,13.7750,NaN,S
10,902,0,3,"Ilieff, Mr. Ylio",male,NaN,0,0,349220,7.8958,NaN,S
378,1270,0,1,"Hipkins, Mr. William Edward",male,55.0,0,0,680,50.0000,C39,S
243,1135,0,3,"Hyman, Mr. Abraham",male,NaN,0,0,3470,7.8875,NaN,S
21,913,0,3,"Olsen, Master. Artur Karl",male,9.0,0,1,C 17368,3.1708,NaN,S
265,1157,0,3,"Lyntakoff, Mr. Stanko",male,NaN,0,0,349235,7.8958,NaN,S


In [11]:
df.shape

(418, 12)

just make a copy of df so the original df not lost
the copy name is df_temp

In [47]:
columns=['Age','Fare','Survived'] # we not use sex because it's categorial and we perform OneHotEncoder
df_temp=df[columns]

In [48]:
df_temp.head(5)

,Age,Fare,Survived
0,34.5,7.8292,0
1,47.0,7.0000,1
2,62.0,9.6875,0
3,27.0,8.6625,0
4,22.0,12.2875,1


In [49]:
df_temp.shape

(418, 3)

In [50]:
df_temp.isnull().sum()

,0
Age,86
Fare,1
Survived,0


HERE calculate the accurary score of model before Discretization.


In [51]:
x=df_temp.drop(columns=['Survived'])
y=df_temp['Survived']

In [52]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [53]:
x_train.head()

,Age,Fare
96,76.0,78.8500
381,26.0,7.8792
89,2.0,23.0000
233,NaN,7.8792
191,NaN,26.0000


In [36]:
x_train.shape

(292, 3)

**NOET**:The change in the number of rows from 418 to 292 in x_train is due to the train_test_split function. When you set test_size=0.3, it means that 30% of the data will be allocated to the test set, while the remaining 70% will be used for the training set.

Here's the breakdown:

Total Rows: 418
Test Size (30%):
0.3
×
418
=
125.4
0.3×418=125.4 (rounded to 125)
Training Size (70%):
0.7
×
418
=
292.6
0.7×418=292.6 (rounded to 293)
Since you specified test_size=0.3, the number of rows in x_train becomes approximately 70% of the original dataset, which is why x_train.shape shows (292, 3).

In [54]:
x_test.shape

(126, 2)

In [55]:
y_train.shape

(292,)

In [56]:
y_test.shape

(126,)

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [58]:
before_dics=DecisionTreeClassifier()

before_dics.fit(x_train,y_train)

y_pred=before_dics.predict(x_test)

accuracy_score(y_test,y_pred)


0.5476190476190477

In [61]:
import numpy as np
from sklearn.model_selection import cross_val_score

In [65]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.5908246225319396

**After performing Discritization**

**Concept of encode='ordinal' or 'onehot**
below

In [72]:
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np

data = np.array([[1], [2], [6], [5], [10]])
kbd = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform')
binned_data = kbd.fit_transform(data)

print(binned_data)
# he binned output will be represented as integers. Each bin will be assigned a unique integer value, starting from 0.


[[0.]
 [0.]
 [1.]
 [1.]
 [2.]]


In [71]:
kbd = KBinsDiscretizer(n_bins=3, encode='onehot', strategy='uniform')
binned_data = kbd.fit_transform(data)

print(binned_data.toarray())  # Convert to a dense array for better readability
# The binned output will be represented as a sparse matrix of binary values (0s and 1s), where each bin corresponds to a column.


[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [90]:
df_temp.isnull().sum()

,0
Age,86
Fare,1
Survived,0


**fresh start for KBinsDiscretizer. because above the model train on missing value as well
but the KBinsDiscretizer not work with missing values**

In [97]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [153]:
columns=['Age','Sex','Fare','Survived']
df_new=df[columns]

In [154]:
df_new.shape

(418, 4)

In [155]:
df_new.iloc[0]

,0
Age,34.5
Sex,male
Fare,7.8292
Survived,0


handle missing values

In [156]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Assuming df_new is already defined with shape (418, 4)

# Define the column transformer
column_transformer = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), [0, 2]),  # Fill Age and Fare with mean
        ('cat', OneHotEncoder(handle_unknown='ignore'), [1])  # One-hot encode Sex
    ],
    remainder='passthrough'  # Keep the other columns unchanged
)

# Apply the transformations
transformed_data = column_transformer.fit_transform(df_new)

# Check the shape of transformed_data
print("Shape of transformed_data:", transformed_data.shape)

# Create a new DataFrame with transformed data
df_transformed = pd.DataFrame(transformed_data)

# Retrieve column names for one-hot encoded features
ohe_columns = column_transformer.named_transformers_['cat'].get_feature_names_out()
print("One-hot encoded columns:", ohe_columns)

# Combine original and new column names
new_columns = ['Age', 'Fare'] + list(ohe_columns) + ['Survived']
print("New column names:", new_columns)

# Set new column names
df_transformed.columns = new_columns

# Replace df_new with the transformed DataFrame
df_new = df_transformed

# Display the transformed DataFrame
print(df_new)


Shape of transformed_data: (418, 5)
One-hot encoded columns: ['Sex_female' 'Sex_male']
New column names: ['Age', 'Fare', 'Sex_female', 'Sex_male', 'Survived']
          Age      Fare  Sex_female  Sex_male  Survived
0    34.50000    7.8292         0.0       1.0       0.0
1    47.00000    7.0000         1.0       0.0       1.0
2    62.00000    9.6875         0.0       1.0       0.0
3    27.00000    8.6625         0.0       1.0       0.0
4    22.00000   12.2875         1.0       0.0       1.0
..        ...       ...         ...       ...       ...
413  30.27259    8.0500         0.0       1.0       0.0
414  39.00000  108.9000         1.0       0.0       1.0
415  38.50000    7.2500         0.0       1.0       0.0
416  30.27259    8.0500         0.0       1.0       0.0
417  30.27259   22.3583         0.0       1.0       0.0

[418 rows x 5 columns]


In [160]:
df_new.isnull().sum()

,0
Age,0
Fare,0
Sex_female,0
Sex_male,0
Survived,0


In [173]:
x=df_new.drop(columns=['Survived'])
y=df_new['Survived']

In [174]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0)

In [175]:
kbinAge=KBinsDiscretizer(n_bins=10, encode='ordinal',strategy='uniform')

In [176]:
trf = ColumnTransformer([
    ('age',kbinAge,[0])
])
# in this we can also perform onehotencoder for sex column

In [177]:
x_train.isnull().sum()

,0
Age,0
Fare,0
Sex_female,0
Sex_male,0


In [179]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf=trf.transform(x_test)

In [184]:
trf.named_transformers_['age'].bin_edges_


array([array([ 0.17 ,  7.753, 15.336, 22.919, 30.502, 38.085, 45.668, 53.251,
              60.834, 68.417, 76.   ])                                       ],
      dtype=object)

In [187]:
output = pd.DataFrame({
    'age': x_train['Age'],
    'age_trf': x_train_trf[:, 0],
})

output['age_labels'] = pd.cut(
    x=x_train['Age'],
    bins=trf.named_transformers_['age'].bin_edges_[0].tolist()
)



In [194]:
output.sample(5)

,age,age_trf,age_labels
166,48.0,6.0,"(45.668, 53.251]"
242,45.0,5.0,"(38.085, 45.668]"
99,33.0,4.0,"(30.502, 38.085]"
62,18.0,2.0,"(15.336, 22.919]"
319,22.0,2.0,"(15.336, 22.919]"


In [199]:
# Fit and transform the training data
x_train_trf = trf.fit_transform(x_train)

# Transform the test data
x_test_trf = trf.transform(x_test)

# Fit the Decision Tree Classifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train_trf, y_train)

# Make predictions on the transformed test data
y_pred2 = dtc.predict(x_test_trf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred2)
print("Accuracy:", accuracy)

Accuracy: 0.47619047619047616
